AG3-Actividad Guiada 3

Manuel Moras Acosta


https://colab.research.google.com/drive/1q4-0wEHn7Rn7JndGYV7SW2r9sti60tCi?usp=sharing

In [ ]:
#!pip install requests

In [1]:
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve('http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp', file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f4de179a940>)

In [3]:
!pip install tsplib95

In [4]:
import tsplib95
import random
from math import e

In [22]:
problem = tsplib95.load(file)

#nodos
Nodos = list(problem.get_nodes())

#aristas
Aristas = list(problem.get_edges())

In [23]:
#Se genera una solucion aleatoria con comienzo en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

In [24]:
#Devuelve la distancia entre dos nodos
def distancia (a,b, problem):
  return problem.get_weight(a,b)

In [8]:
#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion,problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i], solucion[i+1], problem)
  return distancia_total + distancia(solucion[len(solucion)-1],solucion[0], problem)

In [25]:
s=crear_solucion(Nodos)
distancia_total(s,problem)

5154

Búsqueda Aleatoria

In [26]:
def busqueda_aleatoria(problem, N):

  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  mejor_distancia = 10e100

  for i in range(N):
    solucion = crear_solucion(Nodos)
    distancia = distancia_total(solucion, problem)

    if distancia < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia
  
  print("Mejor solucion: ", mejor_solucion)
  print("Distancia   :", mejor_distancia)
  return mejor_solucion

In [27]:
sol = busqueda_aleatoria(problem, 5000)

Mejor solucion:  [0, 13, 34, 16, 19, 18, 26, 1, 33, 36, 20, 21, 25, 38, 22, 5, 35, 31, 17, 14, 23, 6, 12, 37, 32, 41, 10, 4, 28, 9, 39, 29, 8, 24, 40, 11, 2, 7, 15, 30, 3, 27]
Distancia   : 3759


Búsqueda Local. Generador de vecindad

In [65]:
def genera_vecina(solucion, problem):


  mejor_solucion = []
  mejor_distancia = 10e100

  for i in range(1, len(solucion)-1):
    for j in range (i+1, len(solucion)):

      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      distancia_vecina = distancia_total(vecina, problem)

      if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina

  return mejor_solucion

In [66]:
s1=crear_solucion(Nodos)
s2=genera_vecina(s1, problem)
print(distancia_total(s1, problem), s1)
print(distancia_total(s2, problem), s2)

4925 [0, 26, 3, 2, 39, 37, 22, 30, 24, 35, 34, 10, 32, 31, 21, 41, 23, 16, 19, 12, 15, 11, 17, 9, 28, 1, 27, 36, 4, 33, 8, 5, 29, 14, 7, 40, 20, 38, 6, 13, 25, 18]
4380 [0, 26, 3, 2, 39, 40, 22, 30, 24, 35, 34, 10, 32, 31, 21, 41, 23, 16, 19, 12, 15, 11, 17, 9, 28, 1, 27, 36, 4, 33, 8, 5, 29, 14, 7, 37, 20, 38, 6, 13, 25, 18]


In [93]:
import copy
import math
def busqueda_local(problem):
  mejor_solucion=[]

  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = 10e100

  iteracion=0
  while(1):
    iteracion +=1
    vecina = genera_vecina(solucion_referencia, problem)
    distancia_vecina = distancia_total(vecina, problem)

    if distancia_vecina < mejor_distancia:
      mejor_solucion = copy.deepcopy(vecina)
      mejor_distancia = distancia_vecina
    else:
      print("En la iteracion ", iteracion, ", la mejor solucion encontrada es: ", mejor_solucion)
      print("Distancia :", mejor_distancia)
      return mejor_solucion
    
    solucion_referencia = vecina

In [94]:
sol = busqueda_local(problem)

En la iteracion  39 , la mejor solucion encontrada es:  [0, 28, 30, 32, 31, 17, 29, 22, 38, 34, 33, 20, 35, 36, 37, 12, 11, 25, 10, 8, 9, 39, 21, 24, 40, 23, 41, 18, 26, 5, 13, 19, 14, 16, 15, 7, 3, 27, 2, 4, 6, 1]
Distancia : 1703


In [95]:
#generador de vecina aleatoria

def genera_vecina_aleatoria(solucion):
  i,j = sorted(random.sample( range(1, len(solucion)) , 2))

  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

In [96]:
#funcion de probabilidad para determinar si se cambia
# a una solucion peor respecto a la de referencia(exponencial)

def probabilidad(T,d):
  if random.random() <= math.exp(-1*d/ T):
    return True
  else:
    return False

def bajar_temperatura(T):
  return T*.999

In [97]:
#######################################
#Recocido Simulado

def recocido_simulado(problem, TEMPERATURA=1000):

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)

  mejor_solucion =[]
  mejor_distancia = 10e100
  N=0

  while TEMPERATURA > .0001:
    N+=1
    vecina = genera_vecina_aleatoria(solucion_referencia)
    distancia_vecina = distancia_total(vecina,problem)

    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina

    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina)):
      solucion_referencia = copy.deepcopy(vecina)
      distancia_referencia = distancia_vecina

    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solucion encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

In [98]:
sol = recocido_simulado(problem, 1000000)

La mejor solucion encontrada es [0, 1, 3, 4, 6, 26, 18, 12, 11, 25, 8, 22, 38, 30, 28, 17, 36, 35, 31, 20, 33, 34, 32, 27, 2, 29, 9, 39, 24, 40, 21, 23, 41, 10, 5, 13, 19, 14, 16, 15, 37, 7]
con una distancia total de 1601


### Colonia de Hormigas

In [99]:
def Add_Nodo(problem, H ,T ) :
  #Mejora:Establecer una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )


def Incrementa_Feromona(problem, T, H ) :
  #Incrementa segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T ):
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  #Mejora:Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [100]:
def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
  #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  #Mejora: inicializar con valores diferentes dependiendo diferentes criterios
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
  
  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)
  
  
hormigas(problem, 1000)

[0, 2, 17, 31, 13, 6, 1, 7, 16, 37, 20, 24, 40, 32, 10, 29, 34, 14, 5, 15, 23, 35, 36, 3, 21, 22, 8, 41, 30, 19, 39, 25, 9, 12, 11, 18, 4, 28, 38, 26, 27, 33]
3862
